In [ ]:
import pandas as pd
import numpy as np
import pickle
def openpickle(filename):
    with open(filename, "rb") as readfile:
        loaded = pickle.load(readfile)
    return loaded

In [3]:
train_pricing_decisions = pd.read_csv('train_prices_decisions.csv')
train_covariate = openpickle('train_covariate')
train_noisy_embedding = openpickle('train_noisy_embedding')
test_covariate = openpickle('test_covariate')
test_noisy_embedding = openpickle('test_noisy_embedding')
item0_embedding = openpickle('item0embedding')
item1_embedding = openpickle('item1embedding')

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
item_vectors = np.array([item0_embedding, item1_embedding])
train_pricing_decisions = train_pricing_decisions.set_index('user_index')
price_pair = train_pricing_decisions.drop(columns=["item_bought"])

In [ ]:
existing_train_idx = list(train_noisy_embedding.index)
existing_train_covariate = train_covariate.loc[existing_train_idx]
existing_train_embedding = train_noisy_embedding.loc[existing_train_idx]
existing_train_embedding.columns = existing_train_embedding.columns.astype(str)

existing_test_idx = list(test_noisy_embedding.index)
existing_test_covariate = test_covariate.loc[existing_test_idx]
existing_test_embedding = test_noisy_embedding.loc[existing_test_idx]

existing_covariate = pd.concat([existing_train_covariate, existing_test_covariate])
existing_embedding = pd.concat([existing_train_embedding, existing_test_embedding], axis=0)

In [ ]:
new_train_idx = [u for u in list(train_covariate.index) if u not in existing_train_idx]
new_train_covariate = train_covariate.loc[new_train_idx]

new_test_idx = [u for u in list(test_covariate.index) if u not in existing_test_idx]
new_test_covariate = test_covariate.loc[new_test_idx]

In [ ]:
# train KNN to get new user vectors
N_NEIGHBORS = 5
neigh = NearestNeighbors(n_neighbors = N_NEIGHBORS)
neigh.fit(existing_covariate)

neighbor_ids = neigh.kneighbors(new_train_covariate, return_distance=False)
for i in range(len(new_train_idx)):
  curr_embedding = list(existing_embedding.iloc[neighbor_ids[i]].mean(axis=0))
  train_noisy_embedding.loc[new_train_idx[i]] = curr_embedding

neighbor_ids = neigh.kneighbors(new_test_covariate, return_distance=False)
for i in range(len(new_test_idx)):
  curr_embedding = list(existing_embedding.iloc[neighbor_ids[i]].mean(axis=0))
  test_noisy_embedding.loc[new_test_idx[i]] = curr_embedding

train_noisy_embedding = train_noisy_embedding.sort_index()
test_noisy_embedding = test_noisy_embedding.sort_index()

In [ ]:
X = train_covariate.join(train_noisy_embedding @ item_vectors.T).join(price_pair)
X.columns = X.columns.astype(str)
X_test = test_covariate.join(test_noisy_embedding @ item_vectors.T)
X_test.columns = X_test.columns.astype(str)
y = train_pricing_decisions['item_bought']

In [ ]:
X_train, y_train = X, y
print(X_train.shape)

(14000, 7)


In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
clf = LogisticRegressionCV(multi_class="multinomial", max_iter=1000).fit(X_train.values, y_train.values)

Revenue-maximizing prices using gradient descent

In [ ]:
def get_gradient_direction(min0, max0, min1, max1, curr_df):
  res_0, res_1, res_revenue = 0, 0, 0
  for p0 in [min0, max0]:
    for p1 in [min1, max1]:
      curr_revenue = get_curr_revenue(p0, p1, curr_df)
      if curr_revenue > res_revenue:
        res_0 = p0
        res_1 = p1
        res_revenue = curr_revenue
  return res_0, res_1, res_revenue

def get_curr_revenue(p0, p1, curr_df):
  curr_df['price_item_0'] = p0
  curr_df['price_item_1'] = p1
  pred = clf.predict_proba(np.array(curr_df).reshape(1, -1))
  curr_revenue = p0 * pred[0][1] + p1 * pred[0][2]
  return curr_revenue

In [ ]:
item_0_prices, item_1_prices, expected_revenues = [], [], []
max_iter = 100
lr = 0.01
# early_stopping = 3

for k in range(len(X_test)): 
  # setup initial prices (median)
  price0 = train_pricing_decisions.price_item_0.median()
  price1 = train_pricing_decisions.price_item_1.median()
  for num_iter in range(max_iter):
    curr_revenue = get_curr_revenue(price0, price1, X_test.iloc[k].copy())
    min0, max0 = price0 - lr, price0 + lr
    min1, max1 = price1 - lr, price1 + lr
    # use p0 +/- lr, and p1 +/- lr to calculate the combination {p0, p1} that gives the max revenue
    gradient_0, gradient_1, gradient_revenue = get_gradient_direction(min0, max0, min1, max1, X_test.iloc[k].copy())
  
    # if the max revenue from the new {p0, p1} combination is smaller than curr revenue -> curr price for item0 and item1 is maximum
    if gradient_revenue < curr_revenue:
      break
    # else we move to the direction of gradient
    price0 = gradient_0
    price1 = gradient_1
    curr_revenue = gradient_revenue
  item_0_prices.append(price0)
  item_1_prices.append(price1)
  expected_revenues.append(curr_revenue)
  

finished 0 user
finished 500 user
finished 1000 user
finished 1500 user
finished 2000 user
finished 2500 user
